# Examples

Building Nock expressions from scratch can be a challenge, but it is invaluable to understanding how Nock works in practice.  It also helps you understand why compilers make the decisions they do.

## Incrementing a Number

Nock provides only one built-in arithmetic operator:  opcode 4, which adds one to a number.

In [1]:
:subject 41

Subject set to: 41


In [2]:
[4 0 1]

42


That means to grab the value in the subject at address 1 (the entire subject) and increment it directly.

## Decrementing a Number

A decrement operator is more involved.  (The algorithm itself is [a classic problem](https://moronlab.blogspot.com/2010/01/decrement-in-reck.html) in Nock's history.)

The high-level idea is that if you only have the ability to increment and check for equality, then you can build a decrement operator by counting up from zero and checking whether adding one to the current number equals the source number.  If it does, then the current number is one less than the source number, thus its decrement.

We need a few pieces of Nock code to put this together.  Let's call the source number `a` and the other working variable number `b`.

First off, we can pin a constant to start counting from using opcode 1:

```nock
[1 0]
```

We can compare the increment of a number `b` to `a` using opcode 5:

```nock
[5 a [4 b]]
```

(We ignore for a moment how we get `a` and `b`; they will be parts of the subject retrieved with opcode 0 or supplied as constants with opcode 1.)

We need two different paths based on whether the comparison is true or false, mediated by opcode 6.

```nock
[6 [5 a [4 b]] <true-path> <false-path>]
```

The `true-path` is easy:  we just return `b`, since we've found the decrement.  The `false-path` is more involved:  we need to increment `b` and try again.  We can do this by recursively calling our decrement operator with the same `a` and the incremented `b`.  How do we invoke our own operator recursively?  We use an opcode 9 with an opcode 10 to alter the subject for the recursive call.

Finally, we need the ability to pin new variables such as the starting value `b`.  We'll do this using opcode 8, after which point we can refer to the new variable at address 2.

Let's build this piece by piece.

In [4]:
:a [1 43]

Variable 'a' set to: [1 43]


In [5]:
:b [1 0]

Variable 'b' set to: [1 0]


In [9]:
:test [5 a [4 b]]

Variable 'test' set to: [5 [1 43] 4 1 0]


Check the test case (should return false):

In [11]:
test

1


In [13]:
:show test

test = [5 [1 43] 4 1 0]


The source constant `a` is hardcoded as 43, but that's okay for now.  We won't be able to work with `b` as a constant, however, so we'll come back to this expression later.

The next bit we need is how to edit an atom, where `x` is the address of `b` (rather than its value):

```nock
[10 [x 4 b] 0 1]
```

This edits the noun and returns that entire edited noun.  That yields something like this, adjusted against the reference subject:

In [21]:
[10 [1 4 1 0] 0 1]

1


Finally, we need to recurse on the edited value:

```nock
[9 2 10 [x 4 b] 0 1]
```

If opcode 9 is `*[a 9 b c]`, then here `b` is the memory slot of the new subject formed with `c`, which is the edited noun from above.

(We can't demonstrate the opcode 9 in isolation.)

Putting it all together, we get the full decrement operator:

```nock
        [8 [1 6 [5 a [4 2]] 2 [9 2 10 [2 4 2] 0 1]]]
[8 [1 0] 8 [1 6 [5 [1 43] 4 0 6] [0 6] 9 2 10 [6 4 0 6] 0 1] 9 2 0 1]
```

The `[9 2 0 1]` at the tail is the actual evaluation:  from the entire unedited subject, execute the arm at address 2, which is the operator we just built.

In [24]:
[8 [1 0] 8 [1 6 [5 [1 43] 4 0 6] [0 6] 9 2 10 [6 4 0 6] 0 1] 9 2 0 1]

42


That works!  It's still too brittle:  the constant `a` is hardcoded, so we want to make it flexible by supplying the sample positionally rather than as a constant.  We can do that by changing how we pin `a` and `b` at the start and referring to the sample at address 6 via `[0 6]`.

In [25]:
[8 [1 43] 8 [1 0] 8 [1 6 [5 [0 14] 4 0 6] [0 6] 9 2 10 [6 4 0 6] 0 1] 9 2 0 1]

42


## Cores

The idea of a core is that a collection of constant executable Nock formulas can be bundled with references to each other in a consistent layout, allowing for more complex behavior.  That is, if you know where a function is located in a core, you can call it from another function in the same core by referring to its address.  Thus you can compose arbitrarily complex behavior from a set of simpler formulas.

(How to order the functions in a core depends on decisions made by the higher-level language compiler; Hoon, for instance, builds a complex data structure called a `$set` which orders arms by the hash of their names.)

Here is an example of a core which has two constants (42 at address 22 and 43 at address 23) and an executable formula or arm at address 10 that increments its sample:

```nock
[[1 [8 [1 0] [1 4 0 6] 0 1] [1 42] 1 43] 0 1]
```

Note that with two opcode 1 formula pins, we will need to evaluate the core _twice_ to actually run the target arm.  This is why we distinguish an _arm_ (which requires two evaluations with opcode 9) from a _leg_ (which requires only an opcode 0 lookup).

An evaluation of a leg requires pinning the core with opcode 8 then looking up the leg with opcode 0 adjusted for the relative address:

In [ ]:
:subject 43

Subject set to: 43


In [ ]:
[8 [1 [8 [1 0] [1 4 0 6] 0 1] [1 43] 1 43] 0 10]

[1 43]


The leg is invoked with a pair of opcodes 9 and a composited subject:

In [ ]:
[8 [1 [8 [1 0] [1 4 0 6] 0 1] [1 42] 1 43] 8 [9 4 0 1] 9 2 10 [6 7 [0 3] 9 10 0 1] 0 2]

43


You can grab the leg without the third opcode 9, but it doesn't evaluate it so you have to grab the tail of the opcode 1 or evaluate it separately as with an opcode 2.

In [41]:
[8 [1 [8 [1 0] [1 4 0 6] 0 1] [1 42] 1 43] 8 [9 4 0 1] 9 2 10 [6 7 [0 3] 0 21] 0 2]

43


### What's Missing?

Nock is mathematically complete, but it doesn't seem to have many affordances that programmers expect from a language.  Nock leaves some pragmatic elements of programming and the computer environment to its evaluator, or runtime environment.

- Boolean logic (`AND`, `XOR`, `NOT`, etc.) must be implemented out of Nock primitives rather than being axiomatically supplied as operators.
- Side effects (like printing) will be handled by raising special noun patterns to the Nock evaluator.
- Memory is entirely handled by the Nock evaluator.
- Type systems are not part of Nock itself, but can be layered on top of it.
- Evaluation rules are defined, but their implementation is omitted.  You can use a tree-walking interpreter, a bytecode interpreter, or something even more clever to run Nock in practice.  (In fact, you can treat Nock as a spec and not run it at all, as long as you get the same answer!)

* How would you implement the Boolean operators in Nock?  (Properly “loobeans”, since `0` is `TRUE` and `1` is `FALSE` in Nock.)  See [Opcode 6](opcode-6.ipynb) for a starting point.

## Quines

Nock quines are Nock expressions that evaluate to themselves.  They result trivially from [opcode 0](opcode-0.ipynb), which simply looks up a part of the subject.  If the subject is the expression itself, then looking up address 1 returns the entire expression.

In [4]:
:subject [0 1]

Subject set to: [0 1]


This formula will reproduce itself from that subject:

In [5]:
[[0 1] [0 1]]

[[0 1] 0 1]


Other quines are possible as well, and left as an exercise for the reader.